In [13]:
#--------------------
using PowerModels
using Ipopt
using JuMP

#function build_mn_data(base_data; replicates::Int=2)
#    mp_data = PowerModels.parse_file(base_data)
#    return PowerModels.replicate(mp_data, replicates)
#end

# Instancate a Solver
#--------------------

nlp_solver = IpoptSolver(print_level=0)
# note: print_level changes the amount of solver information printed to the terminal


# Load System Data
# ----------------

if VERSION < v"0.7.0"
    powermodels_path = Pkg.dir("PowerModels")
else
    powermodels_path = joinpath(dirname(pathof(PowerModels)), "..")
end

file_name = "ModelData/case5"
file_ext = ".m"
periods = 2
# note: change this string to modify the network data that will be loaded

# load network data
base_data = PowerModels.parse_file(string(file_name,"_",1,file_ext))

# clean load data
for i in keys(base_data["bus"])
    base_data["load"][i] = base_data["load"]["1"]
end

# create multi-network replicates
mp_data = PowerModels.replicate(base_data, periods)

#data 
#data = build_mn_data(string(file_name,"_",1,file_ext),replicates = periods)



# Add zeros to turn linear objective functions into quadratic ones
# so that additional parameter checks are not required
PowerModels.standardize_cost_terms(mp_data, order=2)

# use build_ref to filter out inactive components
#ref = PowerModels.build_ref(data)[:nw][0]
ref = PowerModels.build_ref(mp_data)
# note: ref contains all the relevant system parameters needed to build the OPF model
# When we introduce constraints and variable bounds below, we use the parameters in ref.


###############################################################################
# 1. Building the Optimal Power Flow Model
###############################################################################

# Initialize a JuMP Optimization Model
#-------------------------------------
model = Model(solver = nlp_solver)


# Add Optimization and State Variables
# ------------------------------------

# Add power generation variables for each generator, including bounds
@variable(model, ref[:nw][j][:gen][i]["pmin"] <= pg[j in keys(ref[:nw]), i in keys(ref[:nw][j][:gen])] <= ref[:nw][j][:gen][i]["pmax"])
# Add power flow variables for branches
@variable(model, -ref[:nw][k][:branch][l]["rate_a"] <= p[k in keys(ref[:nw]), (l,i,j) in ref[:nw][k][:arcs]] <= ref[:nw][k][:branch][l]["rate_a"])

# Add Objective Function
# ----------------------

# Minimize cost power generation
# assumes costs are given as quadratic functions
@objective(model, Min,
    sum(gen["cost"][1]*pg[j,i]^2 + gen["cost"][2]*pg[j,i] + gen["cost"][3] for j in keys(ref[:nw]), (i,gen) in ref[:nw][j][:gen])
)


[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 5 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 5 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 2 from -360.0 to -60.0 deg.
[warn

4000 pg[2,4] + 1500 pg[2,2] + 3000 pg[2,3] + 1000 pg[2,5] + 1400 pg[2,1] + 4000 pg[1,4] + 1500 pg[1,2] + 3000 pg[1,3] + 1000 pg[1,5] + 1400 pg[1,1]

In [24]:
#data["nw"][string(1)]["load"][string("5")] = data["nw"][string(1)]["load"][string("1")]
#data["nw"]["1"]["load"]
base_data = PowerModels.parse_file(string(file_name,"_",1,file_ext))

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 5 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 5 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 2 from -360.0 to -60.0 deg.
[warn

Dict{String,Any} with 12 entries:
  "bus"            => Dict{String,Any}("4"=>Dict{String,Any}("zone"=>1,"bus_i"=…
  "source_type"    => "matpower"
  "name"           => "case5"
  "dcline"         => Dict{String,Any}()
  "source_version" => v"2.0.0"
  "gen"            => Dict{String,Any}("4"=>Dict{String,Any}("ncost"=>2,"qc1max…
  "branch"         => Dict{String,Any}("4"=>Dict{String,Any}("br_r"=>0.00108,"r…
  "storage"        => Dict{String,Any}()
  "baseMVA"        => 100
  "per_unit"       => true
  "shunt"          => Dict{String,Any}()
  "load"           => Dict{String,Any}("1"=>Dict{String,Any}("load_bus"=>2,"sta…

In [36]:
for i in keys(base_data["bus"])
    base_data["load"][i] = base_data["load"]["1"]
    println(i)
end

4
1
5
2
3


In [11]:
@variable(model, ref[:nw][j][:gen][i]["pmin"] <= pg[j in keys(ref[:nw]), i in keys(ref[:nw][j][:gen])] <= ref[:nw][j][:gen][i]["pmax"])

┌ Warning: A variable or constraint named pg is already attached to this model. If creating variables programmatically, use the anonymous variable syntax x = @variable(m, [1:N], ...).
└ @ JuMP C:\Users\noahx23\.julia\packages\JuMP\PbnIJ\src\JuMP.jl:852


0 <= pg[j,i] <= .. for all j in {2,1}, i in {..}

In [6]:
ref[:nw][:1][:gen]

Dict{Int64,Any} with 5 entries:
  4 => Dict{String,Any}("pmin"=>0.0,"qc1max"=>0.0,"pg"=>0.0,"model"=>2,"shutdow…
  2 => Dict{String,Any}("pmin"=>0.0,"qc1max"=>0.0,"pg"=>1.7,"model"=>2,"shutdow…
  3 => Dict{String,Any}("pmin"=>0.0,"qc1max"=>0.0,"pg"=>3.2349,"model"=>2,"shut…
  5 => Dict{String,Any}("pmin"=>0.0,"qc1max"=>0.0,"pg"=>4.6651,"model"=>2,"shut…
  1 => Dict{String,Any}("pmin"=>0.0,"qc1max"=>0.0,"pg"=>0.4,"model"=>2,"shutdow…

\begin{align}
%
\mbox{data: } & \nonumber \\
& e^u \mbox{ - energy rating} \\
& sc^u \mbox{ - charge rating} \\
& sd^u \mbox{ - discharge rating} \\
& \eta^c \mbox{ - charge efficiency} \\
& \eta^d \mbox{ - discharge efficiency} \\
& te \mbox{ - time elapsed} \\
& sl \mbox{ - standing losses} \\
& r \mbox{ - injection resistance} \\
& q^l, q^u  \mbox{ - reactive power injection limits} \\
& s^u \mbox{ - thermal injection limit} \\
& i^u \mbox{ - current injection limit} \\
%
\mbox{variables: } & \nonumber \\
& e_i \in (0, e^u) \mbox{ - storage energy at time $i$} \\
& sc_i \in (0, sc^u) \mbox{ - charge amount at time $i$} \\
& sd_i \in (0, sd^u) \mbox{ - discharge amount at time $i$} \\
& S_i \mbox{ - complex bus power injection at time $i$} \\
& I_i \mbox{ - complex bus current injection at time $i$} \\
%
\mbox{subject to: } & \nonumber \\
& e_t - e_s = te \left(\eta^c sc_t - \frac{sd_t}{\eta^d} \right) \label{eq_strg_energy} \\
& sc_t \cdot sd_t = 0 \label{eq_strg_compl} \\
& \Re(S_t) + (sd_t - sc_t) = sl + r |I_t| \label{eq_strg_loss} \\
& q^l \leq \Im(S_t) \leq q^u \label{eq_strg_q_limit} \\
& |S_t| \leq s^u \label{eq_strg_thermal_limit} \\
& |I_t| \leq i^u \label{eq_strg_current_limit}
\end{align}